In [2]:
import pandas as pd
import random
import numpy as np


In [5]:
def initial_solution(df):
    solution = []
    remaining_jobs = df['operation'].tolist()
    while remaining_jobs:
        for job in remaining_jobs:
            predecessors = df.loc[df['operation'] == job, 'predecessors'].values[0]
            if all(predecessor in solution for predecessor in predecessors):
                solution.append(job)
                remaining_jobs.remove(job)
                break
    return solution

def makespan(solution, df):
    # Dummy implementation of makespan calculation
    # You should replace this with your own makespan calculation logic
    return sum(df.loc[df['operation'].isin(solution), 'machine'])

def generate_neighbors(solution, df):
    neighbors = []
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            neighbor = solution[:]
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            if is_feasible(neighbor, df):
                neighbors.append(neighbor)
    return neighbors

def is_feasible(solution, df):
    for i, job in enumerate(solution):
        predecessors = df.loc[df['operation'] == job, 'predecessors'].values[0]
        if any(predecessor in solution[i+1:] for predecessor in predecessors):
            return False
    return True

def TSSA(df, t0, ImprovIter):
    s = initial_solution(df)
    sb = s[:]
    f_sb = makespan(sb, df)
    iter = 0
    T = t0
    tabu_list = []
    L = [s]

    while iter <= ImprovIter:
        iter += 1
        neighbors = generate_neighbors(s, df)

        if not neighbors:
            break

        s_star = min(neighbors, key=lambda sol: makespan(sol, df))
        f_s_star = makespan(s_star, df)
        
        if f_s_star < f_sb or random.random() < np.exp((f_sb - f_s_star) / T):
            s = s_star[:]
            if f_s_star < f_sb:
                sb = s_star[:]
                f_sb = f_s_star
                iter = 0
            L.append(s)
            if s not in tabu_list:
                tabu_list.append(s)

        T = T * 0.95

        if len(tabu_list) > 100:  # Limiting the size of tabu list to 100
            tabu_list.pop(0)

    return sb, f_sb


df = pd.read_csv("sample_15.csv")

# Running TSSA
best_solution, best_makespan = TSSA(df, t0=100, ImprovIter=1000)
print("Best solution:", best_solution)
print("Best makespan:", best_makespan)

KeyboardInterrupt: 